In [2]:
! pip3 install bayesian_optimization
! pip3 install cython

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=395b05ed0d51e647bf9ef7836ad167fd0760ede1a5db3a65379cf0494f934767
  Stored in directory: /home/jovyan/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization
     |████████████████████████████████| 2.0 MB 3.2 MB/s eta 0:00:01


In [1]:
! sudo apt-get update
! sudo apt-get -y install build-essential

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [51.9 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [293 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7,907 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [15.2 kB]
Fetched 8,454 kB in 2s (4,307 kB/s)                   
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu cpp cpp-8 dpkg-dev
  fakeroot g++ g++-8 gcc gcc-8 libalgorithm-diff-perl
  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan5 libatomic1
  libbinutils libc-dev-bin libc6-dev libcc1-0 libdpkg-perl libfakeroot
  libfile-fcntllock-perl libgcc-8-de

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import cython
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%load_ext Cython

In [2]:
train_values_short = pd.read_csv('train_values_short1.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')
test_values_short = pd.read_csv('test_values_short1.csv', index_col='building_id')

In [3]:
%%cython
def hassanat1(double[:] x, double[:] y, double[:] factores):
    cdef int n = x.shape[0]
    cdef double res = 0
    cdef double d, minimo, maximo

    for i in range(n):
        minimo = min(x[i],y[i])
        maximo = max(x[i],y[i])
        if (minimo >=0):
            d =  1 - ( (1+minimo)/(1+maximo) )
        else:
            d =  1 - ( (1+minimo+abs(minimo))/(1+maximo+abs(minimo)) )
        res += d * factores[i]
    
    return res

In [4]:
factores = np.array([1.82264756, 0.14128378, 0.27999595, 0.4755266 , 2.30883141,
       1.02996122, 1.07966807, 1.10292129, 0.83811159, 0.80736305,
       0.74243681, 4.93845144, 4.93845144, 7.76639614, 0.03964061550504756, 0.09684727174529291, 0.025805063269526796])\
        .astype('double')

In [5]:
k = 22
start = time.time()
knn = KNeighborsClassifier(weights="distance",n_neighbors=k,
    metric=hassanat1, metric_params={'factores':factores})
knn.fit(train_values_short, train_labels.values.ravel())

KNeighborsClassifier(metric=<built-in function hassanat1>,
                     metric_params={'factores': array([1.82264756, 0.14128378, 0.27999595, 0.4755266 , 2.30883141,
       1.02996122, 1.07966807, 1.10292129, 0.83811159, 0.80736305,
       0.74243681, 4.93845144, 4.93845144, 7.76639614, 0.03964062,
       0.09684727, 0.02580506])},
                     n_neighbors=22, weights='distance')

In [8]:
ids = test_values_short.index

In [10]:
prediction = knn.predict_proba(test_values_short.loc[ids[:10]])

In [13]:
probas = pd.DataFrame(data=prediction,
                             columns=[1,2,3],
                             index=ids[:10])

In [ ]:
probas.to_csv('probas.csv')